In [1]:
# Standard libraries
import xarray as xr
import numpy as np
import pandas as pd
# import os
# from glob import glob
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.pyplot as plt
%matplotlib inline
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
import seaborn as sns
import iris
from iris.pandas import as_cubes
import sys

from datetime import datetime
from cartopy.util import add_cyclic_point
import gc
import imageio.v2
from IPython import display
# import netCDF4
from global_land_mask import globe
# # Import tobac itself:
import tobac

# Disable a few warnings:
import warnings
warnings.filterwarnings('ignore', category=UserWarning, append=True)
warnings.filterwarnings('ignore', category=RuntimeWarning, append=True)
warnings.filterwarnings('ignore', category=FutureWarning, append=True)
warnings.filterwarnings('ignore',category=pd.io.pytables.PerformanceWarning)

## _Sorting out cells that made it to landfall (used the globe.is_land() library)_

In [2]:
track = pd.read_csv('../saved-files/threshold-15/final-tracks-threshold-15.csv')
track

,frame,idx,hdim_1,hdim_2,num,threshold_value,feature,time,timestr,projection_y_coordinate,projection_x_coordinate,latitude,longitude,cell,time_cell,distance,mean dBZ,max dBZ,area($km^2$)
0,0,2,133.891782,247.588144,6,15,2,2020-03-13 04:00:00,2020-03-13 04:00:00,-2.193892e+06,164588.143590,70.196696,14.290355,2,0 days 00:00:00,125.209617,13.245867,34.567482,8.952468
1,1,5,135.850880,248.149120,2,15,86,2020-03-13 04:05:00,2020-03-13 04:05:00,-2.195851e+06,165149.120397,70.178537,14.301097,2,0 days 00:05:00,123.291260,12.974534,35.577388,20.890860
2,2,5,139.000000,249.440269,2,15,169,2020-03-13 04:10:00,2020-03-13 04:10:00,-2.199000e+06,166440.268555,70.149081,14.328411,2,0 days 00:10:00,119.982671,12.780750,34.063904,21.887120
3,3,4,143.476756,250.870342,6,15,251,2020-03-13 04:15:00,2020-03-13 04:15:00,-2.203477e+06,167870.342039,70.107474,14.356623,2,0 days 00:15:00,115.540119,12.708913,36.584485,22.880821
4,4,5,148.000000,253.920901,3,15,336,2020-03-13 04:20:00,2020-03-13 04:20:00,-2.208000e+06,170920.901252,70.064303,14.426429,2,0 days 00:20:00,110.078668,12.845599,38.092506,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17800,299,4,144.743717,239.312025,4,15,23004,2020-03-14 04:55:00,2020-03-14 04:55:00,-2.204744e+06,156312.025471,70.103689,14.055373,6578,0 days 00:05:00,122.013418,15.240493,41.116443,NaN
17801,300,7,146.461260,240.148842,5,15,23041,2020-03-14 05:00:00,2020-03-14 05:00:00,-2.206461e+06,157148.841602,70.087558,14.073848,6578,0 days 00:10:00,120.179000,14.702386,40.106802,5.970119
17802,298,31,256.000000,174.366873,2,15,22993,2020-03-14 04:50:00,2020-03-14 04:50:00,-2.316000e+06,91366.873305,69.124388,12.259164,6586,0 days 00:00:00,148.655096,15.301029,39.603226,10.944407
17803,299,27,256.815629,176.111734,4,15,23027,2020-03-14 04:55:00,2020-03-14 04:55:00,-2.316816e+06,93111.734429,69.116329,12.301452,6586,0 days 00:05:00,147.098363,15.240493,41.116443,10.943279


In [3]:
track.head()   

,frame,idx,hdim_1,hdim_2,num,threshold_value,feature,time,timestr,projection_y_coordinate,projection_x_coordinate,latitude,longitude,cell,time_cell,distance,mean dBZ,max dBZ,area($km^2$)
0,0,2,133.891782,247.588144,6,15,2,2020-03-13 04:00:00,2020-03-13 04:00:00,-2.193892e+06,164588.143590,70.196696,14.290355,2,0 days 00:00:00,125.209617,13.245867,34.567482,8.952468
1,1,5,135.850880,248.149120,2,15,86,2020-03-13 04:05:00,2020-03-13 04:05:00,-2.195851e+06,165149.120397,70.178537,14.301097,2,0 days 00:05:00,123.291260,12.974534,35.577388,20.890860
2,2,5,139.000000,249.440269,2,15,169,2020-03-13 04:10:00,2020-03-13 04:10:00,-2.199000e+06,166440.268555,70.149081,14.328411,2,0 days 00:10:00,119.982671,12.780750,34.063904,21.887120
3,3,4,143.476756,250.870342,6,15,251,2020-03-13 04:15:00,2020-03-13 04:15:00,-2.203477e+06,167870.342039,70.107474,14.356623,2,0 days 00:15:00,115.540119,12.708913,36.584485,22.880821
4,4,5,148.000000,253.920901,3,15,336,2020-03-13 04:20:00,2020-03-13 04:20:00,-2.208000e+06,170920.901252,70.064303,14.426429,2,0 days 00:20:00,110.078668,12.845599,38.092506,NaN


In [4]:
%%time
cell_landfall = []
for i, j in enumerate(track['cell']):
#     # # Check if a point is on land:
    lat = track['latitude'][i]
    lon = track['longitude'][i]
    is_on_land = globe.is_land(lat, lon)
    if is_on_land:
        # if ((lat >= 69.0) & (lon >= 12.0)):
            # print(f"lat={lat}, lon={lon} is on land: {is_on_land} -- cell_id={j}")
        cell_landfall.append(j)

CPU times: user 1.88 s, sys: 474 µs, total: 1.88 s
Wall time: 3.86 s


In [5]:
%%time
cells_landfall=np.unique(cell_landfall)
cells_landfall

CPU times: user 271 µs, sys: 0 ns, total: 271 µs
Wall time: 275 µs


array([  20,   28,   43,   50,   58,   85,  103,  142,  150,  152,  164,
        182,  202,  206,  234,  241,  249,  251,  254,  278,  281,  299,
        330,  348,  352,  413,  424,  427,  435,  453,  480,  493,  494,
        514,  517,  538,  540,  544,  551,  570,  571,  599,  617,  634,
        635,  636,  644,  645,  646,  648,  649,  657,  671,  689,  711,
        734,  743,  750,  757,  766,  768,  771,  786,  788,  789,  799,
        850,  851,  861,  870,  888,  889,  910,  917,  918,  919,  944,
        949,  952,  963,  976,  990,  991,  996, 1009, 1012, 1015, 1050,
       1070, 1073, 1087, 1103, 1111, 1148, 1165, 1170, 1172, 1183, 1199,
       1228, 1229, 1233, 1240, 1241, 1259, 1290, 1300, 1303, 1326, 1417,
       1425, 1438, 1455, 1469, 1497, 1499, 1504, 1515, 1527, 1536, 1576,
       1587, 1599, 1610, 1616, 1642, 1675, 1678, 1687, 1700, 1706, 1711,
       1742, 1744, 1786, 1802, 1826, 1841, 1852, 1871, 1887, 1902, 1914,
       1924, 1935, 1937, 1942, 1967, 1968, 1973, 19

## _Algorithm to track cells back in time that made it to landfall_

In [6]:
%%time
combined_cells_landfalls = []
for i, cell_ids in enumerate(np.unique(cells_landfall)): # looped through tracks found with AMF1 site
    if cell_ids in track['cell'].values: # checking if those cells(tracks) are within the original tracked dataframe
        # print(f'Cell {cell_ids} present: made it to landfall 😀')
        track_c = track[track['cell'] == cell_ids] # display it if it is within the tracked dataframe
        combined_cells_landfalls.append(track_c) # append the results you get a new dataFrame calld combined_tracks_AMF1
    else:
        print('Cells not present 🥲')
        
# Concatenate the dataframes(combined_tracks_AMD) along the rows axis with keys
combined_cells_landfall_present = pd.concat(combined_cells_landfalls)
combined_cells_landfall_present = combined_cells_landfall_present.reset_index(drop=True)
combined_cells_landfall_present['landfall'] = globe.is_land(combined_cells_landfall_present['latitude'], combined_cells_landfall_present['longitude'])

CPU times: user 324 ms, sys: 16.5 ms, total: 341 ms
Wall time: 675 ms


In [7]:
combined_cells_landfall_present.head()

,frame,idx,hdim_1,hdim_2,num,threshold_value,feature,time,timestr,projection_y_coordinate,projection_x_coordinate,latitude,longitude,cell,time_cell,distance,mean dBZ,max dBZ,area($km^2$),landfall
0,0,20,255.544546,324.527060,17,15,20,2020-03-13 04:00:00,2020-03-13 04:00:00,-2.315545e+06,241527.059669,69.030445,15.954806,20,0 days 00:00:00,23.538329,13.245867,34.567482,23.879002,False
1,1,28,258.825739,324.860030,27,15,109,2020-03-13 04:05:00,2020-03-13 04:05:00,-2.318826e+06,241860.030388,69.000377,15.954579,20,0 days 00:05:00,26.876487,12.974534,35.577388,34.826114,False
2,2,27,262.754401,325.077747,16,15,191,2020-03-13 04:10:00,2020-03-13 04:10:00,-2.322754e+06,242077.747280,68.964541,15.949893,20,0 days 00:10:00,30.868930,12.780750,34.063904,19.899722,False
3,3,30,266.731982,325.601422,8,15,277,2020-03-13 04:15:00,2020-03-13 04:15:00,-2.326732e+06,242601.421660,68.927977,15.952552,20,0 days 00:15:00,34.924637,12.708913,36.584485,15.920343,False
4,4,36,270.335063,326.000000,2,15,367,2020-03-13 04:20:00,2020-03-13 04:20:00,-2.330335e+06,243000.000000,68.894920,15.953109,20,0 days 00:20:00,38.597920,12.845599,38.092506,4.975171,True


In [8]:
len(combined_cells_landfall_present)

4472

## _track up to the time the cell first hit landfall (after it hits the first landfall, remove all the subsequent cells hitting landfall, we are not interested in it)_

In [9]:
%%time
cells_to_first_landfall = []
for i, cell_idx in enumerate(np.unique(cells_landfall)):
    df = combined_cells_landfall_present[combined_cells_landfall_present['cell'] == cell_idx]
    # # find the index of the first occurrence of True
    idx = df.loc[df['landfall']].index[0]
    # slice the DataFrame to keep only the rows up to and including that index
    df = df.loc[:idx]
    # set all subsequent occurrences of True to False
    df.loc[df['landfall'].shift(-1) == True, 'landfall'] = False
    # print(df)
    cells_to_first_landfall.append(df)
first_landfall = pd.concat(cells_to_first_landfall)

CPU times: user 633 ms, sys: 11.9 ms, total: 645 ms
Wall time: 1.98 s


## _calculate relative time --- relative time = time - time_landfall_

In [10]:
%%time
cells_relative_time = []
for index, cell_id in enumerate(np.unique(first_landfall['cell'])):
    if cell_id:
        # print(cell_id)
        cells = first_landfall[first_landfall['cell'] == cell_id] # get all the cell_ids
    # calculate for the relative time ----- relative time = time - time_landfall
        all_cells_times = cells['timestr']; all_cells_times = pd.to_datetime(all_cells_times.values) # convert timestr to datetime
        landfall_time_each_cell = all_cells_times[::-1][0]
        # relative_time = np.absolute(all_cells_times - landfall_time_each_cell) # relative time
        relative_time = np.subtract(landfall_time_each_cell, all_cells_times)
        relative_time = relative_time.seconds // 60
        # print(cell_id, relative_time)
        cells_relative_time.append(relative_time)

CPU times: user 551 ms, sys: 2.05 ms, total: 553 ms
Wall time: 1.67 s


In [11]:
cells_relative_time[:20]

[Int64Index([20, 15, 10, 5, 0], dtype='int64'),
 Int64Index([0], dtype='int64'),
 Int64Index([0], dtype='int64'),
 Int64Index([0], dtype='int64'),
 Int64Index([0], dtype='int64'),
 Int64Index([20, 15, 10, 5, 0], dtype='int64'),
 Int64Index([0], dtype='int64'),
 Int64Index([0], dtype='int64'),
 Int64Index([30, 25, 20, 15, 10, 5, 0], dtype='int64'),
 Int64Index([0], dtype='int64'),
 Int64Index([45, 40, 35, 30, 25, 20, 15, 10, 5, 0], dtype='int64'),
 Int64Index([0], dtype='int64'),
 Int64Index([0], dtype='int64'),
 Int64Index([0], dtype='int64'),
 Int64Index([0], dtype='int64'),
 Int64Index([0], dtype='int64'),
 Int64Index([0], dtype='int64'),
 Int64Index([0], dtype='int64'),
 Int64Index([0], dtype='int64'),
 Int64Index([0], dtype='int64')]

In [12]:
# cells_relative_time
cells_relative_times = pd.Series([item for sublist in cells_relative_time for item in sublist])
# first_landfall['relative_time'] = cells_relative_times
# cells_relative_times.values
first_landfall['relative_time'] = cells_relative_times.values

In [13]:
len(first_landfall)

2049

In [14]:
first_landfall.to_csv('../saved-files/threshold-15/all-cells-first_landfall-threshold-15.csv', index=False)

In [15]:
# final = first_landfall[['cell', 'latitude', 'longitude', 'distance', 'max dBZ','mean dBZ', 'area' ,'timestr', 'landfall']]
# final

# END 